# Technical TEST
## Parul 
## parulgandass25@gmail.com

To import the data from the provided URL and perform the requested analysis, We are using a Jupyter Notebook with Python and SQL capabilities. 

Below code retrieves the data from the provided URL using Pandas, filters out the countries with more than 2 million cases, and then creates a SQLite in-memory database to store the filtered data. It then performs an SQL query to group the data by WHO Region and Country, calculating the rolling 7-day (weekly) window for cases and deaths.

In [1]:
%matplotlib widget
import pandas as pd
import numpy as np 
import sqlite3
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.graph_objs as go

In [2]:
#Let's start by retrieving the data and creating the necessary table.
# Reading the data from the URL
url = "https://covid19.who.int/WHO-COVID-19-global-data.csv"
data = pd.read_csv(url)

#List of countries with more than 2 million cases in total
country_set = list(set(data[data['Cumulative_cases'] > 2e6]['Country']))

# data of countries with more than 2 million cases in total 
df = data[data['Country'].isin(country_set)]
df = df.reset_index(drop=True)

In [3]:
# Creating a SQLite in-memory database and table "covid_data" that groups the data by WHO Region and Country and 
#generates a rolling 7 day window of cases and deaths, for those countries with more than 2 million cases in total.
conn = sqlite3.connect(":memory:")
df.to_sql("covid_data", conn, index=False)


56626

In [4]:
# Defining the SQL query to group data by WHO Region and Country with a rolling 7-day (weekly) window
query = """
SELECT
    Country,
    WHO_region,
    min(Date_reported) Report_WeekStart,
    max(Date_reported) Report_WeekEnd,
    SUM(New_cases) as total_new_cases_7_day_window,
    SUM(New_deaths) as total_new_deaths_7_day_window,
    MAX(Cumulative_cases) as Cumulative_cases,
    MAX(Cumulative_deaths) as Cumulative_deaths
FROM covid_data
GROUP BY WHO_region,
    Country,strftime('%W', Date_reported), strftime('%Y', Date_reported)
ORDER BY Country, strftime('%Y', Date_reported), strftime('%W', Date_reported);
"""

In [5]:
# Execute the query and retrieve the results
results = pd.read_sql_query(query, conn)

In [6]:
#below is the content of the new table created
results

,Country,WHO_region,Report_WeekStart,Report_WeekEnd,total_new_cases_7_day_window,total_new_deaths_7_day_window,Cumulative_cases,Cumulative_deaths
0,Argentina,AMRO,2020-01-03,2020-01-05,0,0,0,0
1,Argentina,AMRO,2020-01-06,2020-01-12,0,0,0,0
2,Argentina,AMRO,2020-01-13,2020-01-19,0,0,0,0
3,Argentina,AMRO,2020-01-20,2020-01-26,0,0,0,0
4,Argentina,AMRO,2020-01-27,2020-02-02,0,0,0,0
...,...,...,...,...,...,...,...,...
8275,Viet Nam,WPRO,2023-04-17,2023-04-23,12704,1,11543060,43187
8276,Viet Nam,WPRO,2023-04-24,2023-04-30,16802,1,11559862,43188
8277,Viet Nam,WPRO,2023-05-01,2023-05-07,14069,8,11573931,43196
8278,Viet Nam,WPRO,2023-05-08,2023-05-14,15636,5,11589567,43201


# Data Exploration

## Chart 1: Cases vs Deaths for Countries with more than 2 million cases

In [7]:
# !pip3 install ipywidgets --user
# ! pip3 install ipympl --user
# conda install -c conda-forge ipympl

In [8]:
# ! jupyter nbextension enable --py --sys-prefix widgetsnbextension
# ! jupyter nbextension install --py --symlink --sys-prefix ipympl

In [9]:
import sys
sys.executable

'/Users/parulgandass/anaconda3/bin/python'

In [10]:
!conda list ipympl

# packages in environment at /Users/parulgandass/anaconda3:
#
# Name                    Version                   Build  Channel
ipympl                    0.9.3              pyhd8ed1ab_0    conda-forge


In [11]:

# Generate the chart line chart for Cases vs Deaths for Countries with more than 2 million cases

country_list = country_set + ['All']    #list of countries with more than 2 million cases (options for user to select from)

#defining a function to generate the chart
def chart(country):
    
    fig = plt.figure(figsize=(10, 6))
    # if customer doesn't want the info about all countries, show data for selected countries only
    if 'All' not in country:
        df1 = df[df['Country'].isin(country)]              #filtering data for selected countries only
        
    else:
        df1 = df                                           #else show data about all countries
        
    #generating the plot by countries with labels and legends
    for group,records in df1.groupby('Country'):
        plt.plot(records['Cumulative_cases'],records['Cumulative_deaths'], label = group)
    plt.xlabel('Cases (7-day window)')
    plt.ylabel('Deaths (7-day window)')
    plt.title('COVID-19: Cases vs Deaths (Countries with >2M cases)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    
#use below dropdown to select countries. Multiple values can be selected with shift and/or ctrl (or command) 
#pressed and mouse clicks or arrow keys.
multi = widgets.SelectMultiple(
    options= sorted(country_list),
    value=['All'],
    description='Country',
#     disabled=False
)

widgets.interact(chart, country = multi)



interactive(children=(SelectMultiple(description='Country', index=(0,), options=('All', 'Argentina', 'Australi…

<function __main__.chart(country)>

The above chart helps compare the relationship between cases and deaths differs across different countries with more than 2 million cases. It provides an overview of the mortality rate and allows for identifying trends. Each line represents a country, and the position of each point on the line indicates the total number of cases and total corresponding deaths. The lines are colored based on their respective Country. The slop of the line gives information about the mortality rate. Lines closer to x-axis shows lower number of deaths with increasing number of cases whereas countries with line closer to y-axis were heavily affected by COVID-19 with highest death rate.

Let us take example of 2 countries Italy and Japan for better understanding. This visualization provides insights into how severly both the countries were affected. As dipicted above, Line graph for Japan is much below Italy, which reveals that Japan have experienced a significant number of cases but relatively low mortality, indicating potential differences in healthcare systems, demographics, or other factors influencing outcomes. 
The vertical grid lines indicates difference in total number of death with same number of total cases in both the countries. Following the third vertical grid line, when total cases reported for both Japan and Italy had reached approx 10,000,000 at some point of time, the corresponding total deaths were approx 30000 and 145000respectively.

Note - Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys. Select Italy and Japan for refrence.


The line plot is effective in representing the correlation between cases and deaths for individual countries. It helps identify countries that deviate from the overall trend and provides insights into the severity of the situation in different nations. 


## Chart 2: Compares Cases vs Deaths by WHO Region

In [13]:
data['Month_Year'] = data.Date_reported.apply(lambda x: x[:7])
monthly = data.groupby(['WHO_region','Month_Year']).agg({'New_cases':'sum','New_deaths':'sum'}).reset_index()
count = data.groupby(['Country','WHO_region']).agg({'New_cases':'sum','New_deaths':'sum'}).reset_index()
allreg = data.groupby(['WHO_region']).agg({'New_cases':'sum','New_deaths':'sum'}).reset_index()


def stack(Region, Type):
    fig = plt.figure(figsize=(10, 6))
    # if region = all, use data grouped by region for the bar plot
    if Region == 'ALL':
        dt = allreg
        y = 'WHO_region'       
        
    #else use data on for selected region and group by based in requirement (country or timeperiod/month)
    else:
        if Type == 'Country':
            dt = count[count['WHO_region']==Region]
            y = 'Country'
        else:
            dt = monthly[monthly['WHO_region']==Region]
            y = 'Month_Year'
        
    dt = dt[[y,'New_cases','New_deaths']]

    ax = fig.add_subplot(111) # Create matplotlib axes
    ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.


    dt.New_cases.plot(x=y, kind='bar', color='pink', ax=ax, width=0.4,label = 'Total Cases', position=1)
#     if Region == 'ALL' or Type == 'Country':
    dt.New_deaths.plot(x=y, kind='bar', color='cyan', ax=ax2, width=0.4,label = 'Total Deaths', position=0)
        
#     else:
#     dt.New_deaths.plot(x=y, color='blue', ax=ax2, label = 'Total Deaths')

    ax.set_ylabel('Cases')
    ax2.set_ylabel('Deaths')
    
    ax.set_xticklabels(dt[y])
    
    # ask matplotlib for the plotted objects and their labels
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)
    plt.title(' Comparing Cases and Deaths by ' + Region + ' Region')
    plt.show()
    
#button for selection of tyoe of distribution for each region data   
ty = widgets.ToggleButtons(
    options=['Time', 'Country'],
    description='Distribution',
    tooltips=['Total New Cases/Deaths in each Country', 'Total Cases/Deaths each month'],

)
    
widgets.interact(stack, Region = (['ALL'] + list(set(data['WHO_region']))) ,Type = ty)
# stack('EMRO','New')

interactive(children=(Dropdown(description='Region', options=('ALL', 'AFRO', 'AMRO', 'SEARO', 'EURO', 'EMRO', …

<function __main__.stack(Region, Type)>

This visualization allows for a regional comparison of the relationship between cases and deaths. The bar chart provides a visual representation of the total number of COVID-19 cases reported and deaths by WHO Region. It allows for easy comparison between regions, identify regions that have been more severely impacted by the virus and provides insights into the effectiveness of regional response strategies. It helps identify regions that may have a higher or lower death rate relative to the number of cases reported. The color coding facilitates the comparison between cases and deaths and enables us to identify potential patterns or discrepancies.

The initial bar graph represents total number or cases and deaths as 2 combined bars for each region, with cases on left scale and deaths on right scale. Reading the above gragh, it is safe to say that EURO was the most spread region with highest number of reported cases whereas AMRO appears to be most affected with highest number of deaths. The difference between the hight of pink and blue bar for AFRO, EMRO and SEARO appears to be similar, representing they have had similar impact in terms deaths relative to the number of cases. Based on the above visualisation WPRO seem to have manages the situation best with lowest mortality rate (highest positive gap between two bars).

We also have option to have a closer look at each region. Selecting a particular region with Time distribution, the bar allows us to observe any regional pattern/variations with time in the impact of COVID-19. Let's compare the cases vs deaths distribution for SEARO region with time. The first Covid-19 wave appears to hit the region May 2020 till Jan 2021 with highest impact during september 2020. later, the impact of second wave was sudden and sever during the mid of year 2021 with highest number of cases and deaths. Moving on, the thrid wave hit the region during early 2022 but region appears to have effective response strategies resulting in comparatively lower mortality rate.

The visualisation also provides insights into the country-wise distribution within a region. For example, within EURO region, France and germany have had the highest number of cases, But the positive difference between the pink and blue bar for both countries shows their effective response strategies and with comparatively low mortality rate. Whereas, Russia Federation have the highest death rate with maximum negative differnce between pink and blue bar.

Note: 
    
    Posotive difference - Pink bar higher than Blue bar
      
    Negative difference - Blur bar higher than Pink bar

## Why each chart type might or might not be better than the others in each situation

The choice between chart types depends on the specific analysis objectives and the information you want to convey. Let's discuss the suitability of each chart type in the given situations:

- **Cases vs Deaths for Countries with more than 2 million cases**: The line plot is effective in representing the relationship between cases and deaths for individual countries (correlation between two variables). It helps identify countries that deviate from the overall trend and provides insights into the severity of the situation in different nations. 

    Line graphs provide an excellent way to map dependent variables that are both quantitative. When both variables are quantitative, the line segment that connects two points on the graph expresses a slope, which can be interpreted visually relative to the slope of other lines. This chart type allows for direct comparison between the number of cases and deaths for individual countries.

    Multiple line graphs have space-saving characteristics over a comparable grouped bar graph. Because the data values are marked by small marks (points) and not bars, they do not have to be offset from each other. 
    
    The bar chart might become cluttered with the large number of countries. It might also be challenging to visualize the trends for each country.
    
. 

- **Compare Cases vs Deaths by WHO Region**: This double axis bar plot with allows for a comparison of cases vs death and the overall impact of COVID-19 across different regions. It helps identify regional patterns and differences in the relationship between cases and deaths with time.

    Bar graphs are better for comparing larger changes or differences in data among groups. Bar graphs are best to compare things between different groups or to track changes over time. A line chart's primary variable is also numeric and continuous (WHO rehion is our scenario) which shows the connection between data points in one line instead of bars.

    This chart type enables comparison between different WHO regions, providing an overview of the distribution of cases and deaths across regions. It can help identify regions with higher case or death counts and detect any regional disparities.

Choosing the appropriate chart type depends on the nature of the data and the specific insights you want to convey. Both chart types are suitable for their respective situations because they emphasize different aspects of the data. The first chart focuses on individual countries, highlighting outliers and individual country characteristics, while the second chart emphasizes regional differences and patterns.

## Chart 3: Case Fatality Rate (CFR) for each Country

In [21]:

#groupby data by country and get total case count and deaths 
df1 = data.groupby(['Country']).agg({'New_cases':'sum','New_deaths':'sum'}).reset_index()

#Calculate the case fatality rate (CFR) for each country by 
#dividing the total deaths by the total cases and multiplying by 100.
df1['Case Fatality Rate'] = (df1['New_deaths'] / df1['New_cases']) * 100

#renaming column names
df1 = df1.rename(columns={'New_cases': 'Total Cases', 'New_deaths': 'Total Deaths'})


#function to plot chloropleth map
def chloro(Type, View):
    choropleth_map = go.Figure(
        data = {
            'type':'choropleth',
            'locations':df1['Country'],
            'locationmode':'country names',
            'colorscale':'rdbu',
            'z':df1[Type],
            'colorbar':{'title':Type},
            'marker': {
                'line': {
                    'color':'rgb(255,255,255)',
                    'width':2
                }
            }
        },     
        layout = {      
          'geo':{
              'projection':{
                  'type': View 
              },
              'scope':'world', 
          }  
        })
    return choropleth_map

#radio button for the type of View options
view = widgets.RadioButtons(
    options=['orthographic','equirectangular'],
    description='View'

)
    
widgets.interact(chloro, Type = ['Case Fatality Rate','Total Cases','Total Deaths'], View = view)


interactive(children=(Dropdown(description='Type', options=('Case Fatality Rate', 'Total Cases', 'Total Deaths…

<function __main__.chloro(Type, View)>

This visualization highlights countries with higher mortality rates relative to their total case counts and reveal geographical patterns in fatality rates. We can identify countries with the highest death rates in relation to their total number of cases. It helps identify countries where the virus has had a higher impact on mortality compared to the number of cases. It showcases countries where the mortality rate has been particularly high compared to the number of cases. It draws attention to the countries facing challenges in managing the impact of the virus on public health.

We have two options to view the geographical area. Countries are coloured relative to the fatality rate. Dark blue colour represents high fatality rate, as the colour lightens converting to red fatality rate goes dowm. Darker the red, lowere the fataality rate of the country. From the above visualisation, it is safe to say Yemen is one of the highest fatality rate country whereas, China comparatively has very low fatality rate. Comparing India and Australia, Australia is darker red than India, depicting lowere fatality rate than India.

We can also highligh countries based on total case counts or deaths using the above widget. USA appears to have highest number of case count followed by China. USA also have the highest number of death count followed by Brazil. It shows the variation in the total number of cases and deaths across different countries, providing insights into the geographical spread of the pandemic.


## Chart 4: Comparing Cases and Deaths with time period for each Country

In [19]:
# a rolling 7 day window of cases and deaths, for those countries with more than 2 million cases in total
results = results.rename(columns={'total_new_cases_7_day_window':'New_cases', 'total_new_deaths_7_day_window':'New_deaths'})

# Generate the chart based on requirements
def line(Country, Type, Data):
    
#     plt.figure(figsize = (20,8))
    plt.figure(figsize=(10, 6))
    dt = results[results['Country']==Country]
    plt.plot(dt['Report_WeekEnd'],dt[Type+'_'+Data.lower()], label = Country)
        
    plt.xticks(rotation = 90)
    plt.xlabel('Time period')
    plt.ylabel(Data)
    plt.title('COVID-19: No of ' + Data + ' over the period of time in ' +Country)
    plt.grid(True)
    plt.legend()
    plt.show()


#button for selection of type of info (New or Cumulative)
t = widgets.ToggleButtons(
    options=['New','Cumulative'],
    description='Type'

)


#button for selection of required data (cases or death)
d = widgets.RadioButtons(
    options=['Cases','Deaths'],
    description='Data'

)

widgets.interact(line, Country = sorted(country_set), Type = t, Data = d)

interactive(children=(Dropdown(description='Country', options=('Argentina', 'Australia', 'Austria', 'Banglades…

<function __main__.line(Country, Type, Data)>

This chart can help identify the periods of surges or declines in new or total cases/deaths and observe the overall trend for each country. It can provide insights into the effectiveness of containment measures and the impact of vaccination campaigns.

Having a deeper look at the situation in India over the period, we can see 3 surges/waves. With 2nd wave resulting in the highest spread. Zoomin in on the first wave, it hit India during mid May 2020 with highest cases recorded during mid Sep 2020. Second wave has had the highest impact, with highest number cases recorded in 1st week of May 2021.

Looking at the death count for India, second wave appears to have the most impact with highest number of deaths. Whereas, even though third wave has had quite high number of cases reported, India appears to have effective response strategies resulting in comparatively lower mortality rate.

Note: 

    use the squar box (second button from bottom) in the left hand vertical tool bar widget to zoom in, click home button to zoom out.

# Conclusion
## The data is very interesting

- The data is interesting because it allows us to analyze the impact of COVID-19 across countries and regions. We can observe the variations in the number of cases and deaths and identify the most affected areas. By comparing the cases and deaths, we can gain insights into the severity of the disease and its impact on different parts of world.

- This data reveals countries that have experienced a significant number of cases but relatively low mortality (or vice versa), indicating potential differences in healthcare systems, demographics, or other factors influencing outcomes.

- The provided COVID-19 data offers valuable insights into the global impact of the virus. These visualizations provide different perspectives on the data and can help understand the trends, patterns, and variations in the COVID-19 situation across countries and WHO regions.

- The data shows the variation in the total number of cases across different countries, providing insights into the geographical spread of the pandemic. It draws attention to the countries facing challenges in managing the impact of the virus on public health.

- It allows us to compare the impact of the virus across different regions. It helps identify regions with higher or lower mortality rates relative to the number of cases, providing insights into potential variations in healthcare infrastructure, policies, and demographics.

- By examining the data from multiple perspectives and visualizing it in different ways, we can gain a deeper understanding of the global COVID-19 situation and identify areas where further investigation or intervention may be needed.

# THANK YOU